In [2]:
from time import time
import pyopencl as cl
import numpy
import os
import math


In [3]:
#codigo original em https://www.ime.usp.br/~pf/algoritmos/aulas/mrgsrt.html

In [ ]:
#implementação 1

In [32]:
#Vector size
ORDER = 32768

# Max dim for NDRange
DIM = 1

# number of times to do each ordenation
COUNT = 10

N = ORDER

#criar um vetor de valores inteiros aleatorios
h_V = numpy.random.random(ORDER).astype(numpy.float32) * 10000
h_V = h_V%1000

def error(N, V):
    err = 0;
    for i in range(N-1):
        if V[i] > V[i+1]:
            return 1
    return 0

def results(N, C, run_time):
    print(run_time, "seconds")
    errsq = error(N, C)
    if (errsq > 0):
        print("Wrong ordenation:")
h_V

array([708.4087  , 325.688   , 754.18604 , ...,  96.211426, 103.79126 ,
       178.82837 ], dtype=float32)

In [33]:
# Setup OpenCL
context = cl.create_some_context()
print("DEVICE", context.devices)

queue = cl.CommandQueue(context)

d_V = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_V)
d_V_aux = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_V.nbytes)
d_V_result = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_V.nbytes)

DEVICE [<pyopencl.Device 'GeForce RTX 2060 SUPER' on 'NVIDIA CUDA' at 0x1d417ec6b30>]


In [34]:
kernelsource = open("merge.cl").read()
print("<<<KERNEL SOURCE>>>\n", kernelsource)

program = cl.Program(context, kernelsource).build()
merge_sort_gpu = program.merge_sort_gpu
merge_sort_gpu.set_scalar_arg_dtypes([numpy.int32, None, None, None])
print("\n===== OpenCL, mergesort of a float vector with size ", N, "======\n")

# Do the multiplication COUNT times
for i in range(COUNT):
    start_time = time()
    merge_sort_gpu(queue, (N,), None, N, d_V, d_V_aux, d_V_result)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_V, d_V_result)
    results(N, h_V, run_time)
    


<<<KERNEL SOURCE>>>
 /*
	Fernando Sassi Nunes
	Codigo orinal do merge sort obtido em :	https://www.ime.usp.br/~pf/algoritmos/aulas/mrgsrt.html
	primeira implementaÃ§Ã£o do Merge Sort paralela
*/

static void intercala (int p, int q, int r, float v[], float w[])
{
   int i, j;
   for (i = p; i < q; ++i) w[i] = v[i];
   for (j = q; j < r; ++j) w[r+q-j-1] = v[j];				//escreve na ordem inversa
   i = p; j = r-1;
   for (int k = p; k < r; ++k)
      if (w[i] <= w[j]) v[k] = w[i++];
      else v[k] = w[j--];
}


__kernel void merge_sort_gpu(
			 const int    N,
    __global       float* R,
	__global       float* w,
	__global       float* V)
{
	
    int p = get_global_id(0);
	//escreve em v os valores de R
	V[p] = R[p];
																//barreira para garantir que todo o vetor foi copiado
	barrier(CLK_LOCAL_MEM_FENCE);
	
    int current_size = 1;
	while (current_size < N) {
      if (p + current_size < N && (p)%(2*current_size) == 0) {	//mantem apenas as threads que sÃ£o potencias de 2, 
    

In [7]:
h_V


array([7.812500e-03, 4.296875e-02, 8.496094e-02, ..., 9.999580e+02,
       9.999756e+02, 9.999785e+02], dtype=float32)

In [29]:
#implementação 2

#Vector size
ORDER = 32768

# Max dim for NDRange
DIM = 1

# number of times to do each ordenation
COUNT = 10

N = ORDER

#criar um vetor de valores inteiros aleatorios
h_V = numpy.random.random(ORDER).astype(numpy.float32) * 10000
h_V = h_V%1000

def error(N, V):
    err = 0;
    for i in range(N-1):
        if V[i] > V[i+1]:
            return 1
    return 0

def results(N, C, run_time):
    print(run_time, "seconds")
    errsq = error(N, C)
    if (errsq > 0):
        print("Wrong ordenation:")
h_V

array([405.84766 ,  25.07251 ,  30.884277, ..., 209.97314 , 833.3645  ,
       694.2969  ], dtype=float32)

In [30]:
# Setup OpenCL
context = cl.create_some_context()
print("DEVICE", context.devices)

queue = cl.CommandQueue(context)

d_V = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_V)
d_V_aux = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_V.nbytes)
d_V_result = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_V.nbytes)

DEVICE [<pyopencl.Device 'GeForce RTX 2060 SUPER' on 'NVIDIA CUDA' at 0x1d417ec6b30>]


In [31]:
kernelsource = open("merge2.cl").read()
print("<<<KERNEL SOURCE>>>\n", kernelsource)

#numero maximo de threads usadas
num_threads = 2048

program = cl.Program(context, kernelsource).build()
merge_sort_gpu = program.merge_sort_gpu
merge_sort_gpu.set_scalar_arg_dtypes([numpy.int32, None, None, None, numpy.int32])
print("\n===== OpenCL, mergesort of a float vector with size ", N, " and with ",num_threads, " threads " , "======\n")


# Do the multiplication COUNT times
for i in range(COUNT):
    start_time = time()
    merge_sort_gpu(queue, (N,), None, N, d_V, d_V_aux, d_V_result, num_threads)
    queue.finish()
    run_time = time() - start_time
    cl.enqueue_copy(queue, h_V, d_V_result)
    results(N, h_V, run_time)
    

<<<KERNEL SOURCE>>>
 /*
	Fernando Sassi Nunes
	Codigo orinal do merge sort obtido em :	https://www.ime.usp.br/~pf/algoritmos/aulas/mrgsrt.html
	Segunda implementaÃ§Ã£o do Merge Sort paralela
*/

static void intercala (int p, int q, int r, float v[], float w[])
{
   int i, j;
   for (i = p; i < q; ++i) w[i] = v[i];
   for (j = q; j < r; ++j) w[r+q-j-1] = v[j];			//escreve na ordem inversa
   i = p; j = r-1;
   for (int k = p; k < r; ++k)
      if (w[i] <= w[j]) v[k] = w[i++];
      else v[k] = w[j--];
}

void mergesort (int n, float v[], float w[])			//mergesort original
{
   int b = 1;
   while (b < n) {
      int p = 0;
      while (p + b < n) {
         int r = p + 2*b;
         if (r > n) r = n;
         intercala (p, p+b, r, v, w);
         p = p + 2*b; 
      }
      b = 2*b;
   }
}

__kernel void merge_sort_gpu(
			 const int    N,
    __global       float* R,
	__global       float* w,
	__global       float* V,
			 const int num_threads)
{
    int p = get_global_id(0);
	//escreve

In [25]:
h_V

array([1.0009766e-02, 6.6406250e-02, 6.7138672e-02, ..., 9.9987952e+02,
       9.9994824e+02, 9.9995508e+02], dtype=float32)